In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 2.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30315 sha256=db3f57fe19b5d1b023737d76800561471f01fce8ec78cb561fd4d7ee68061c98
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=253b0d5819f4a25fdcf41f91a908d64b4a391961b482b13018d9e5c662ce27c9
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=01cc1eb215ab2558f39835ceb4bc52aa748ac739ae6c0849fd55ca464a46f1b5
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 3.2MB/s 


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import keras.backend as K
import bert

In [3]:
import pandas as pd 
import numpy as np 
import re
import math
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv('/content/drive/My Drive/parto tech/sentiment_analysis/twitter/train.csv') 

In [5]:
def remove_at_and_hashtag (text) :
  text = text.replace('@user',' ')
  return text.replace('#',' ')
  #TODO : use hashtags

In [7]:
def preprocess_text(sen):
    sentence = remove_at_and_hashtag(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [8]:
train['tweet'] = train['tweet'].apply(preprocess_text)

In [9]:
reviews = train.tweet.to_list()

In [10]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [18]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [19]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [13]:
y = np.array(train.label)
reviews_with_len = [[review, y[i], len(review)] for i, review in enumerate(tokenized_reviews)]

In [14]:
reviews_with_len.sort(key=lambda x: x[2])

In [15]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [16]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [68]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [69]:
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [99]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn1_filters=16,
                 cnn2_filters=32,
                 cnn3_filters=64,
                 dnn_units=256,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn1_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn2_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn3_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation=tf.nn.leaky_relu)
        self.dense_2 = layers.Dense(units=dnn_units, activation=tf.nn.leaky_relu)
        self.dropout_1 = layers.Dropout(rate=dropout_rate)
        self.dropout_2 = layers.Dropout(rate=dropout_rate)

        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) 
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout_1(concatenated)
        concatenated = self.dense_2(concatenated)
        concatenated = self.dropout_2(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [83]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN1_FILTERS = 16
CNN2_FILTERS = 32
CNN3_FILTERS = 64
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 6

In [100]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn1_filters=CNN1_FILTERS,
                        cnn2_filters=CNN2_FILTERS,
                        cnn3_filters=CNN3_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [40]:
def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [101]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=[get_f1])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [102]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/6
900/900 [==============================] - 68s 76ms/step - loss: 0.1374 - get_f1: 0.4451
Epoch 2/6
900/900 [==============================] - 67s 75ms/step - loss: 0.0311 - get_f1: 0.7946
Epoch 3/6
900/900 [==============================] - 68s 75ms/step - loss: 0.0086 - get_f1: 0.8743
Epoch 4/6
900/900 [==============================] - 68s 75ms/step - loss: 0.0049 - get_f1: 0.8825
Epoch 5/6
900/900 [==============================] - 68s 75ms/step - loss: 0.0021 - get_f1: 0.8903
Epoch 6/6
900/900 [==============================] - 68s 75ms/step - loss: 0.0036 - get_f1: 0.8897
